# 5-3 | 트리의 앙상블 |

In [82]:
#앙상블 학습 중의 랜덤 포레스트(Random Forest)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine-date')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42) 

**랜덤 포레스트**

- 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만듬 (부트스트랩 샘플)

------------------------------------

In [83]:
from sklearn.model_selection import cross_validate #교차 검증함수(훈련세트를 섞어 폴드를 나주지 않아 미리 섞거나 분할기를 지정해야함)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42) 
scores = cross_validate(rf, train_data, train_target, return_train_score=True, n_jobs=-1) #교차검증 수행 함수
print(scores)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

{'fit_time': array([0.20345497, 0.20445323, 0.20644736, 0.19547534, 0.20046401]), 'score_time': array([0.02792501, 0.02792573, 0.02792668, 0.03291154, 0.02792501]), 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]), 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}
0.9973541965122431 0.8905151032797809


In [84]:
rf.fit(train_data, train_target)
print(rf.feature_importances_) #특성의 일부를 랜덤하게 선택하여 훈련하기 때문에 일반화 성능을 높이는데 도움이 됨

[0.23167441 0.50039841 0.26792718]


In [85]:
# 랜덤 포레스트에서 부트스트랩 샘플에 포함되지 않고 남은 샘플들을 이용하여 결정트리를 평가할수 있음 OOB샘플(Out Of Bag)
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_data, train_target)
print(rf.oob_score_)

0.8934000384837406


**엑스트라 트리**

- 각 결정 트리를 만들 때 전체 훈련 세트를 사용함(부트스트랩 샘플 사용 안함)
- 엑스트라 트리의 회귀 버전은 "ExtraTreesRegressor"

------------------------------------

In [86]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_data, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [87]:
et.fit(train_data, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


**그레이디언트 부스팅**

- 깊이가 얇은 결정 트리를 사용하여 이진트리의오차를 보완하는 방식으로 앙상블 하는 방법
- "GradientBoostingClassifier" 기본적으로 깊이 3인 결정트리 100개 사용 -> 깊이가 얇기떄문에 과대적합에 강하고 일반화 성능을 기대할 수 있음

------------------------------------

In [88]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_data, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [89]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_data, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [90]:
gb.fit(train_data, train_target)
print(gb.feature_importances_)

[0.15872278 0.68011572 0.16116151]


**히스토그램 기반 그레이디언트 부스팅**

- 정형 데이터를다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘임
- 입력 특성을 256개의 구간으로 나눠 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음
- "HistGradientBoostingClassifier"
------------------------------------

In [91]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_data, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [92]:
hgb.fit(train_data, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [93]:
hgb.score(test_data, test_target)

0.8723076923076923